In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import datasets
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from tqdm import tqdm


In [16]:
# Hyperparameters
# num_classes = 10
# learning_rate = 0.001
# batch_size = 64
# num_epochs = 5
# optimizer = optim.Adam
# activation_func = nn.ReLU()
# dropout_rate = 0.1
# no_neurons = 128
# criterion = nn.CrossEntropyLoss()
# kernel_size=3
# stride=1
# padding=1
# number_of_filters0=32
# number_of_filters1=32
# length_of_input0=32



class CNN_3_class(nn.Module):
    def __init__(self, num_classes = 10
                    ,kernel_size=3
                    ,stride=1
                    ,padding=1
                    ,number_of_filters0=32
                    ,number_of_filters1=32
                    ,length_of_input0=32
                    ,no_neurons = 128):
        super(CNN_3_class, self).__init__()
        self.conv1 = nn.Conv2d(3, number_of_filters0, kernel_size, stride, padding)
        self.pool1 = nn.MaxPool2d(2)
        length_of_input1 = self.Conv2d_output_size(length_of_input0, kernel_size, stride, padding)/2
        self.conv2 = nn.Conv2d(number_of_filters0, number_of_filters1, kernel_size, stride, padding)
        self.pool2 = nn.MaxPool2d(2)
        length_of_input2 = self.Conv2d_output_size(length_of_input1, kernel_size, stride, padding)/2
        self.fc1 = nn.Linear(int(number_of_filters1*length_of_input2*length_of_input2), no_neurons)
        self.fc2 = nn.Linear(no_neurons, num_classes)
        # parameters
        self.num_classes = num_classes
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.number_of_filters0 = number_of_filters0
        self.number_of_filters1 = number_of_filters1
        self.length_of_input0 = length_of_input0
        self.no_neurons = no_neurons


    def Conv2d_output_size(self, w, k, s, p):
        '''
        w - width of input image
        k - kernel size
        s - stride
        p - padding
        '''
        return (w - k + 2 * p) / s + 1
        
    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        length_of_input1 = self.Conv2d_output_size(self.length_of_input0, self.kernel_size, self.stride, self.padding)/2
        x = self.pool2(torch.relu(self.conv2(x)))
        length_of_input2 = self.Conv2d_output_size(length_of_input1, self.kernel_size, self.stride, self.padding)/2
        x = x.view(-1, int(self.number_of_filters1*length_of_input2*length_of_input2))
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def train_step(self, data, optimizer, criterion):
        x, y = data

        optimizer.zero_grad()

        logits = self(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        accuracy = (logits.argmax(dim=1) == y).float().mean()

        return {'loss': loss, 'accuracy': accuracy}

    def test_step(self, data, criterion):
        x, y = data

        logits = self(x)
        loss = criterion(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()

        return {'loss': loss, 'accuracy': accuracy}

In [17]:

test_loader = datasets.val_loader
train_loader = datasets.train_loader
augmentation1_loader = datasets.augmentation1_loader
augmentation2_loader = datasets.augmentation2_loader

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Załaduj dane CIFAR10
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])
cifar10_train = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
cifar10_test = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

# Utwórz DataLoader dla danych trenujących i testowych
trainloader = torch.utils.data.DataLoader(cifar10_train, batch_size=4, shuffle=True)
testloader = torch.utils.data.DataLoader(cifar10_test, batch_size=4, shuffle=False)

# Załaduj model AlexNet z wytrenowanymi wagami na zbiorze ImageNet
model = models.alexnet(pretrained=True)

# Zmień rozmiar ostatniej warstwy klasyfikatora na 10 wyjść
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 10)

# Przenieś model na kartę graficzną, jeśli jest dostępna
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Zdefiniuj funkcję straty i optymalizator
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Trenuj model
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print("Epoch %d, loss: %.3f" % (epoch + 1, running_loss / len(trainloader)))

# Oblicz metryki na danych testowych
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in testloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Oblicz dokładność i inne metryki
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="macro")
recall = recall_score(y_true, y_pred, average="macro")
f1 = f1_score(y_true, y_pred, average="macro")


Files already downloaded and verified
Files already downloaded and verified


c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
print("Accuracy: %.3f" % accuracy)
print("Precision: %.3f" % precision)
print("Recall: %.3f" % recall)
print("F1 score: %.3f" % f1)

In [18]:
learning_rate = 0.001
# batch_size = 64 TEGO NIE UZYWAMY A JESLI CHCEMY TO W LOADERZE TRZEBA DODAC
num_epochs = 2
optimizer = optim.Adam
activation_func = nn.ReLU()
# dropout_rate = 0.1
criterion = nn.CrossEntropyLoss()

# Create model, criterion, and optimizer
model = CNN_3_class(num_classes = 10,
                    kernel_size=3,
                    stride=1,
                    padding=1,
                    number_of_filters0=64,
                    number_of_filters1=64,
                    length_of_input0=32,
                    no_neurons = 16)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
epochs = num_epochs
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    train_losses = []
    train_accuracies = []
    for data in tqdm(train_loader):
        results = model.train_step(data, optimizer, criterion)
        train_losses.append(results['loss'].item())
        train_accuracies.append(results['accuracy'].item())

    # Calculate average training loss and accuracy for the epoch
    avg_train_loss = sum(train_losses) / len(train_losses)
    avg_train_accuracy = sum(train_accuracies) / len(train_accuracies)
    print(f'Train loss: {avg_train_loss:.4f}, Train accuracy: {avg_train_accuracy:.4f}')

    # Test the model
    val_losses = []
    val_accuracies = []
    with torch.no_grad():
        for data in tqdm(test_loader):
            results = model.test_step(data, criterion)
            val_losses.append(results['loss'].item())
            val_accuracies.append(results['accuracy'].item())

    # Calculate average test loss and accuracy for the epoch
    avg_validation_loss = sum(val_losses) / len(val_losses)
    avg_validation_accuracy = sum(val_accuracies) / len(val_accuracies)
    print(f'Validation loss: {avg_validation_loss:.4f}, Validation accuracy: {avg_validation_accuracy:.4f}')


Epoch 1/2


  2%|▏         | 25/1250 [00:02<02:20,  8.73it/s]


KeyboardInterrupt: 

### CNN from article about weighted random search

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import datasets
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from tqdm import tqdm

class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=736, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=736, out_channels=508, kernel_size=3, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=508, out_channels=664, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=664, out_channels=916, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv5 = nn.Conv2d(in_channels=916, out_channels=186, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=186, out_channels=352, kernel_size=3, padding=1)
        self.linear = nn.Linear(in_features=22528, out_features=1229)
        self.output = nn.Linear(in_features=1229, out_features=num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.maxpool1(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.maxpool2(x)
        x = self.conv5(x)
        x = nn.functional.relu(x)
        x = self.conv6(x)
        x = nn.functional.relu(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        x = nn.functional.relu(x)
        x = self.output(x)
        return x



    def train_step(self, data, optimizer, criterion):
        x, y = data

        optimizer.zero_grad()

        logits = self(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        accuracy = (logits.argmax(dim=1) == y).float().mean()

        return {'loss': loss, 'accuracy': accuracy}

    def test_step(self, data, criterion):
        x, y = data

        logits = self(x)
        loss = criterion(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()

        return {'loss': loss, 'accuracy': accuracy}

In [25]:
test_loader

In [2]:
from torchvision import models
import torch
alexnet = models.alexnet(pretrained=True)
 

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Import Pillow
from PIL import Image
img = Image.open("Cifar10/val/1.png")

In [7]:
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

In [8]:
img_t = transform(img)
batch_t = torch.unsqueeze(img_t, 0)

In [9]:
	
alexnet.eval()
	
out = alexnet(batch_t)
print(out.shape)

torch.Size([1, 1000])


In [12]:
# First, load the model
resnet = models.resnet101(pretrained=True)
 
# Second, put the network in eval mode
resnet.eval()
 
# Third, carry out model inference
out = resnet(batch_t)
 
# Forth, print the top 5 classes predicted by the model
_, indices = torch.sort(out, descending=True)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to C:\Users\mikol/.cache\torch\hub\checkpoints\resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:03<00:00, 46.5MB/s] 


NameError: name 'labels' is not defined

In [10]:
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

FileNotFoundError: [Errno 2] No such file or directory: 'imagenet_classes.txt'

In [11]:
_, index = torch.max(out, 1)
 
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
 
print(labels[index[0]], percentage[index[0]].item())

NameError: name 'labels' is not defined